In [1]:
# Import statements
import pandas as pd # database loading and manipulation library
import numpy as np # numeric python library
import matplotlib.pyplot as plt # plotting and graphing library
import operator
%matplotlib notebook

In [3]:
currentSpring2020 = pd.read_csv('2020-spring-match-data-OraclesElixir-2020-02-24.csv', low_memory=False)

In [60]:
#my way of computing the player's value to the team during the game
def KDAScore(data):
    return data['k']+data['d']+data['a']

def TimeScore(data):
    return \
    (data['csdat10']+data['csdat15'])*50 + \
    data['gdat10'] + \
    data['gdat15'] + \
    data['xpdat10']*.3

def TimeScore2016(data):
    return \
    data['csdat10']*50 + \
    data['gdat10'] + \
    data['gdat15'] + \
    data['xpdat10']*.1

# Calculate an individual player's score
def PlayerScore(data):    
    return \
        data['KDA'] + \
        (pd.to_numeric(data['fb'], errors='coerce') + data['fbassist'])*500 + \
        data['earnedgpm']*2 + \
        data['cspm']*3 + \
        data['wpm'] + \
        data['wcpm']*100 + \
        data['dmgtochampsperminute']*3.4 + \
        data['teamdragkills']*20 + \
        data['heraldkills']*50 + \
        data['elders']*100 + \
        pd.to_numeric(data['fd'], errors='coerce')*100 + \
        data['fbaron']*20 + \
        data['tScores']

In [61]:
cleanCurrent = currentSpring2020.loc[(currentSpring2020['league'] == 'LCS') | (currentSpring2020['league'] == 'LEC')]
cleanCurrent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 420 to 3023
Data columns (total 98 columns):
gameid                     1200 non-null object
url                        1200 non-null object
league                     1200 non-null object
split                      1200 non-null object
date                       1200 non-null object
week                       1200 non-null float64
game                       1200 non-null int64
patchno                    1200 non-null float64
playerid                   1200 non-null int64
side                       1200 non-null object
position                   1200 non-null object
player                     1200 non-null object
team                       1200 non-null object
champion                   1200 non-null object
ban1                       1200 non-null object
ban2                       1200 non-null object
ban3                       1200 non-null object
ban4                       1200 non-null object
ban5                       

In [62]:
cleanCurrent.loc[:, "KDA"] = KDAScore(cleanCurrent)
cleanCurrent.loc[:, "tScores"] = TimeScore(cleanCurrent)
cleanCurrent.loc[:, 'PlayerScore'] = PlayerScore(cleanCurrent)
current_player_score = cleanCurrent.groupby("player")["PlayerScore"].mean()

In [63]:
cleanCurrent['player'].unique()

array(['Bwipo', 'Selfmade', 'Nemesis', 'Rekkles', 'Hylissang', 'Alphari',
       'Xerxe', 'Nukeduck', 'Upset', 'Destiny', 'Team', 'Wunder',
       'Jankos', 'Perkz', 'Caps', 'Mikyx', 'Orome', 'Shadow', 'Humanoid',
       'Carzzy', 'Kaiser', 'Sacre', 'Trick', 'Jenax', 'Crownshot',
       'LIMIT', 'Cabochard', 'Skeanz', 'Saken', 'Comp', 'Jactroll',
       'Finn', 'Inspired', 'Larssen', 'Hans Sama', 'VandeR', 'Dan Dan',
       'Razork', 'Febiven', 'Bvoy', 'Denyk', 'Expect', 'Caedrel',
       'Mickey', 'Patrik', 'Tore', 'Odoamne', 'Gilius', 'Abbedagge',
       'FORG1VEN', 'Dreams', 'Licorice', 'Blaber', 'Nisqy', 'Zven',
       'Vulcan', 'Impact', 'Shernfire', 'Jensen', 'Doublelift', 'CoreJJ',
       'Ruin', 'Wiggily', 'Crown', 'Stixxay', 'Smoothie', 'Huni', 'Grig',
       'Froggen', 'Johnsun', 'aphromoo', 'Ssumday', 'Meteos', 'Ryoma',
       'Cody Sun', 'Stunt', 'Hauntzer', 'Closer', 'Goldenglue', 'FBI',
       'Keith', 'sOAZ', 'Xmithie', 'Eika', 'Altec', 'Hakuho', 'V1per',
       'Santori

In [64]:
ML = ['Mad Lions','Orome', 'Shadow', 'Humanoid', 'Carzzy', 'Kaiser'] 
Excel = ['Excel','Caedrel','Expect','Mickey','Patrik','Tore'] 
VitJ = ['Vitality J','Cabochard', 'Jactroll','Skeanz', 'Saken', 'Comp'] 
VitS = ['Vitality S','Steeelback','Cabochard','Skeanz', 'Saken', 'Comp']
Msf = ['Misfits','Dan Dan','Razork', 'Febiven', 'Bvoy', 'Denyk']
S04wFG = ['Schalke 04 wFG','Abbedagge', 'Odoamne', 'Dreams','FORG1VEN','Gilius']
S04wFL = ['Schalke 04 wFL','Abbedagge', 'Odoamne', 'Dreams','FORG1VEN','Lurox']
S04aF = ['Schalke 04 aF','Abbedagge', 'Odoamne', 'Dreams','Innaxe','Lurox']
Rogue = ['Rogue','Finn', 'Inspired', 'Larssen', 'Hans Sama', 'VandeR']
SK = ['SK Gaming','Sacre', 'Trick', 'Jenax', 'Crownshot', 'LIMIT']
Fnc = ['Fnatic','Bwipo', 'Selfmade', 'Nemesis', 'Rekkles', 'Hylissang']
OG = ['Origen','Nukeduck','Alphari','Xerxe','Upset', 'Destiny']
G2 = ['G2','Wunder', 'Jankos', 'Caps', 'Perkz', 'Mikyx']

In [65]:
teamScoresLEC = { }

def ComputeTeamScoreLEC(scores, teams):
    total = 0
    team = teams[1:]
    for player in team:
        total += scores[player]
    teamScoresLEC[teams[0]] = total

ComputeTeamScoreLEC(current_player_score, SK)
ComputeTeamScoreLEC(current_player_score, OG)
ComputeTeamScoreLEC(current_player_score, Excel)
ComputeTeamScoreLEC(current_player_score, Rogue)
ComputeTeamScoreLEC(current_player_score, ML)
ComputeTeamScoreLEC(current_player_score, Fnc)
ComputeTeamScoreLEC(current_player_score, G2)
ComputeTeamScoreLEC(current_player_score, Msf)
ComputeTeamScoreLEC(current_player_score, S04wFG)
ComputeTeamScoreLEC(current_player_score, S04wFL)
ComputeTeamScoreLEC(current_player_score, S04aF)
ComputeTeamScoreLEC(current_player_score, VitJ)
ComputeTeamScoreLEC(current_player_score, VitS)

In [66]:
sorted_teamScoresLEC = sorted(teamScoresLEC.items(), key=operator.itemgetter(1), reverse = True)
sorted_teamScoresLEC

[('Origen', 11634.158347638222),
 ('G2', 11550.463571660222),
 ('Rogue', 11541.506004548499),
 ('Misfits', 11166.089564282778),
 ('Excel', 10189.017296504298),
 ('Fnatic', 9959.321749120443),
 ('Schalke 04 aF', 8890.786578306277),
 ('Vitality S', 7843.680292188223),
 ('Mad Lions', 7808.1971605533),
 ('Schalke 04 wFG', 7490.833035968111),
 ('Schalke 04 wFL', 7399.738291399777),
 ('Vitality J', 6764.5567342115555),
 ('SK Gaming', 6327.6567030012)]

|  Model        |    Actual     |    Preseason  |  Personal    |
|:-------------:|:-------------:|:-------------:|:------------:|
| 1. Origen     | 1. Origen     | 1. G2 Esports | 1. G2 Esports|
| 2. G2 Esports | 1. Fnatic     | 2. Origen     | 2. Origen    |
| 3. Rogue      | 1. G2 Esports | 3. Fnatic     | 3. Misfits   |
| 4. Misfits    | 1. Misfits    | 4. SK Gaming  | 4. Fnatic    |
| 5. Excel      | 5. Rogue      | 5. Scahlke 04 | 5. Rogue     |
| 6. Fnatic     | 5. Mad Lions  | 6. Rogue      | 6. Mad Lions |
| 7. Scahlke 04 | 7. Excel      | 7. Excel      | 7. Excel     |
| 8. Vitality   | 8. SK Gaming  | 8. Vitality   | 8. Scahlke 04|
| 9. Mad Lions  | 8. Scahlke 04 | 9. Misfits    | 9. Vitality  |
|10. SK Gaming  |10. Vitality   |10. Mad Lions  |10. SK Gaming |


Notes:
* Clumps: Origen = G2 = Rogue = Misfits > Excel = Fnatic > S04 > Vitality = Mad Lions > SK
* Scahlke is way better without Forgiven
* Vitality is a lot better with Steeelback
* Mad Lions is a sloppy team

In [56]:
DIG = ['Dignitas','Huni', 'Grig', 'Froggen', 'Johnsun', 'aphromoo']
EG = ['Evil Geniuses','Svenskeren', 'Bang', 'Zeyzal', 'Kumo', 'Jiizuke']
TSM = ['Team Solomid','Bjergsen', 'Broken Blade', 'Kobbe', 'Biofrost', 'Dardoch']
IMT = ['Immortals','sOAZ', 'Xmithie', 'Eika', 'Altec', 'Hakuho']
GGS = ['Golden Guardians','Hauntzer','FBI','Keith','Goldenglue','Closer']
TLS = ['Team Liquid S','Impact', 'Shernfire', 'Jensen', 'Doublelift', 'CoreJJ']
TLB = ['Team Liquid B','Impact', 'Broxah', 'Jensen', 'Doublelift', 'CoreJJ']
FLY = ['Fly Quest','V1per', 'Santorin', 'PowerOfEvil', 'WildTurtle', 'IgNar']
C9 = ['Cloud 9','Licorice', 'Blaber', 'Nisqy', 'Zven','Vulcan']
CLGC = ['Counter Logic Gaming C','Ruin', 'Wiggily', 'Crown', 'Stixxay', 'Smoothie']
CLGP = ['Counter Logic Gaming P','Ruin', 'Wiggily', 'Pobelter', 'Stixxay', 'Smoothie'] 
T100 = ['100 Thieves','Ssumday', 'Meteos', 'Ryoma', 'Cody Sun', 'Stunt']

In [57]:
cleanCurrent['player'].unique()

array(['Bwipo', 'Selfmade', 'Nemesis', 'Rekkles', 'Hylissang', 'Alphari',
       'Xerxe', 'Nukeduck', 'Upset', 'Destiny', 'Team', 'Wunder',
       'Jankos', 'Perkz', 'Caps', 'Mikyx', 'Orome', 'Shadow', 'Humanoid',
       'Carzzy', 'Kaiser', 'Sacre', 'Trick', 'Jenax', 'Crownshot',
       'LIMIT', 'Cabochard', 'Skeanz', 'Saken', 'Comp', 'Jactroll',
       'Finn', 'Inspired', 'Larssen', 'Hans Sama', 'VandeR', 'Dan Dan',
       'Razork', 'Febiven', 'Bvoy', 'Denyk', 'Expect', 'Caedrel',
       'Mickey', 'Patrik', 'Tore', 'Odoamne', 'Gilius', 'Abbedagge',
       'FORG1VEN', 'Dreams', 'Licorice', 'Blaber', 'Nisqy', 'Zven',
       'Vulcan', 'Impact', 'Shernfire', 'Jensen', 'Doublelift', 'CoreJJ',
       'Ruin', 'Wiggily', 'Crown', 'Stixxay', 'Smoothie', 'Huni', 'Grig',
       'Froggen', 'Johnsun', 'aphromoo', 'Ssumday', 'Meteos', 'Ryoma',
       'Cody Sun', 'Stunt', 'Hauntzer', 'Closer', 'Goldenglue', 'FBI',
       'Keith', 'sOAZ', 'Xmithie', 'Eika', 'Altec', 'Hakuho', 'V1per',
       'Santori

In [58]:
teamScoresLCS = { }

def ComputeTeamScoreLCS(scores, teams):
    total = 0
    team = teams[1:]
    for player in team:
        total += scores[player]
    teamScoresLCS[teams[0]] = total

ComputeTeamScoreLCS(current_player_score, TSM)
ComputeTeamScoreLCS(current_player_score, C9)
ComputeTeamScoreLCS(current_player_score, T100)
ComputeTeamScoreLCS(current_player_score, GGS)
ComputeTeamScoreLCS(current_player_score, DIG)
ComputeTeamScoreLCS(current_player_score, IMT)
ComputeTeamScoreLCS(current_player_score, TLS)
ComputeTeamScoreLCS(current_player_score, TLB)
ComputeTeamScoreLCS(current_player_score, EG)
ComputeTeamScoreLCS(current_player_score, CLGC)
ComputeTeamScoreLCS(current_player_score, CLGP)
ComputeTeamScoreLCS(current_player_score, FLY)

In [59]:
sorted_teamScoresLCS = sorted(teamScoresLCS.items(), key=operator.itemgetter(1), reverse = True)
sorted_teamScoresLCS

[('Cloud 9', 17524.3568166882),
 ('Team Solomid', 12294.807993497554),
 ('Team Liquid S', 10318.528557957367),
 ('Team Liquid B', 10016.28598058845),
 ('Golden Guardians', 9622.393861175),
 ('Evil Geniuses', 9095.9564609832),
 ('Dignitas', 7215.444392576333),
 ('Counter Logic Gaming P', 6982.8283202795),
 ('100 Thieves', 6482.648016479874),
 ('Fly Quest', 6327.886446664499),
 ('Counter Logic Gaming C', 5306.653446911375),
 ('Immortals', 4977.9046149589)]

|  Model             |    Actual          |    Preseason       |  Personal          |
|:------------------:|:------------------:|:------------------:|:------------------:|
| 1. Cloud 9         | 1. Cloud 9         | 1. Cloud 9         | 1. Cloud 9         |
| 2. Team Solomid    | 2. Fly Quest       | 2. Team Liquid     | 2. Team Solomid    |    
| 3. Team Liquid     | 3. Team Solomid    | 3. Dignitas        | 3. Fly Quest       |
| 4. Golden Guardians| 4. Immortals       | 4. CLG             | 4. Dignitas        |
| 5. Evil Geniuses   | 4. Dignitas        | 5. Evil Geniuses   | 5. Evil Geniuses   |
| 6. Dignitas        | 6. Evil Geniuses   | 6. Team Solomid    | 6. Golden Guardians|
| 7. CLG             | 6. Team Liquid     | 7. Fly Quest       | 7. Team Liquid     |
| 8. 100 Thieves     | 6. Golden Guardians| 8. Golden Guardians| 8. Immortals       |
| 9. Fly Quest       | 6. 100 Thieves     | 9. Immortals       | 9. 100 Thieves     |
|10. Immortals       |10. CLG             |10. 100 Thieves     |10. CLG             |

Notes:
* Clumps: Cloud 9 >> TSM > Team Liquid = Golden Guardians = Evil Geniuses >> Dignitas = CLG Pob = 100 Thieves = Fly Quest >> Immortals
* This model is bad at handling good team synergy with "ok" players
* CLG with Crown over Pobelter puts them on the same level as Immortals

In [36]:
currentLCS = cleanCurrent.loc[cleanCurrent['league'] == 'LCS']
currentLCS_player_score = currentLCS.groupby("player")["PlayerScore"].mean()
currentLCS_player_score_sort = sorted(currentLCS_player_score.items(), key=operator.itemgetter(1), reverse = True)
currentLCS_player_score_sort

[('Team', 8603.620396325989),
 ('Zven', 4773.504050163499),
 ('FBI', 4386.383000810375),
 ('Licorice', 4308.825697550001),
 ('Doublelift', 3417.4062589285995),
 ('Froggen', 3256.376098359778),
 ('Blaber', 3166.2075456539005),
 ('Bjergsen', 3081.0586804218888),
 ('PowerOfEvil', 3067.1854536240003),
 ('Broken Blade', 3015.212595938),
 ('Kobbe', 2943.325175188556),
 ('Bang', 2863.656610479001),
 ('Jensen', 2840.2144883348),
 ('Impact', 2778.1387880998),
 ('Nisqy', 2660.9359555381006),
 ('Jiizuke', 2496.0148969129),
 ('Pobelter', 2486.757518301),
 ('Vulcan', 2352.8835677826996),
 ('Cody Sun', 2107.721426899625),
 ('Eika', 1925.2941407478),
 ('Wiggily', 1676.2876242519),
 ('Huni', 1674.0715118047779),
 ('Kumo', 1660.2895178843003),
 ('Biofrost', 1645.12253447),
 ('Goldenglue', 1620.63768070875),
 ('Dardoch', 1512.4445630346665),
 ('Santorin', 1477.9351187356249),
 ('Hauntzer', 1468.5000971712498),
 ('Meteos', 1429.838627908125),
 ('Ruin', 1411.7730323706),
 ('Closer', 1408.153731945625),
 (